In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
pn.extension()

In [2]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters  # VideoOutputModeConfig, InteractivePlaceCellConfigs
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences


In [3]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSession
from neuropy.core.epoch import NamedEpoch
from neuropy.utils.misc import print_seconds_human_readable

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

neuropy module not found, adding directory to sys.path. 
 >> Updated sys.path.


In [ ]:
# KDiba Old Format:
sess = DataSessionLoader.kdiba_old_format_session(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')
active_sess_config = sess.config
session_name = sess.name

In [41]:
sess.epochs.to_dataframe()

,start,stop,label,duration
0,0.000000,1739.153364,maze1,1739.153364
1,1739.153364,1932.420005,maze2,193.266641


In [42]:
## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
# active_epoch_name = 'maze1'
# active_epoch_name = 'maze2'

# active_epoch = NamedEpoch(
#         name=active_epoch_name,
#         start_end_times=sess.epochs[active_epoch_name]
#     )

active_epoch_name = 'maze'
active_epoch = NamedEpoch(
        name=active_epoch_name,
        start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]]
    )
# Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
# 20996/932598 total spikes spanning 40/64 units remain in subsession

active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right

## Get the config corresponding to this epoch/session settings:
# active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch)
active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch,
                        video_output_config=VideoOutputModeConfig(active_frame_range=np.arange(100.0, 120.0), video_output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name), active_is_video_output_mode=False),
                        plotting_config=PlottingConfig(output_subplots_shape=active_subplots_shape, output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name))) # '3|1    
## Looks like the only real outputs are: active_epoch_session_Neurons, active_epoch_pos
active_config.plotting_config.active_output_parent_dir.mkdir(parents=True, exist_ok=True) # makes the directory if it isn't already there

# Get the filtered and time-constrainted session and position data for this epoch:
active_epoch_session = sess.get_neuron_type('pyramidal').time_slice(active_epoch.t_start, active_epoch.t_stop)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
active_epoch_session_Neurons, active_epoch_pos, active_epoch_position_times = active_epoch_session.neurons, active_epoch_session.position, active_epoch_session.position.time

Constraining to units with type: pyramidal
Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
20996/932598 total spikes spanning 40/64 units remain in subsession


In [43]:
# print(active_epoch_session.epochs.to_dataframe())
# print(sess.spikes_df) # max is in 4.2305 x 10^4
# print(sess.position.to_dataframe()) # max is in 4.2305 x 10^4
# print(sess.flattened_spiketrains.spikes_df) # t_seconds is in the same units as the position dataframe, which is NOT seconds, and 't' is a factor of 1x10^4 larger than 't_seconds'
# # Get only the rows with a lap != -1:
# # spikes_df = spikes_df[(spikes_df.lap != -1)] # 229887 rows × 13 columns
# # spikes_df # 932598 rows × 14 columns

In [44]:
## Compute Placefields if needed:
from neuropy.analyses import Pf1D, Pf2D, perform_compute_placefields, plot_all_placefields
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True
should_display_2D_plots = False
# Setup the placefield computation config
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=3, grid_bin=2, smooth=2)
try: active_epoch_placefields1D
except NameError: active_epoch_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_epoch_session_Neurons, active_epoch_pos, active_config.computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=True)
# Plot the placefields computed and save them out to files:
if should_display_2D_plots:
    ax_pf_1D, occupancy_fig, active_pf_2D_figures = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config)
else:
    print('skipping 2D placefield plots')
    
# Focus on the 2D placefields:
active_epoch_placefields = active_epoch_placefields2D
# Get the cell IDs that have a good place field mapping:
good_placefield_neuronIDs = np.array(active_epoch_placefields.ratemap.neuron_ids) # in order of ascending ID
print('good_placefield_neuronIDs: {}; ({} good)'.format(good_placefield_neuronIDs, len(good_placefield_neuronIDs)))

## Filter by neurons with good placefields only:
active_epoch_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
active_epoch_session_Neurons = active_epoch_session.neurons

## Plotting Colors:
pf_sort_ind = np.array([int(i) for i in np.arange(len(active_epoch_session.neuron_ids))]) # convert to integer scalar array
pf_colors = get_neuron_colors(pf_sort_ind) # colors are by ascending index ID

active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colors.T # Make the colormap from the listed colors, used seemingly only by 'runAnalysis_PCAandICA(...)'
active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)


Recomputing active_epoch_placefields...
	 done.
Recomputing active_epoch_placefields2D...
	 done.
skipping 2D placefield plots
good_placefield_neuronIDs: [ 2  3  4  5  7  9 11 12 17 18 21 22 23 27 28 29 33 34 38 39 42 44 45 46
 47 48 53 55 57 58]; (30 good)


In [45]:
# active_sess_laps_bak = sess.laps
active_epoch_session.laps = sess.laps
# active_epoch_session.flattened_spiketrains.spikes_df
# active_epoch_session.laps
# active_epoch_session.laps.lap_start_stop_time goes from 190.4446 to 5607.2581, and looks like it would match the value on my "curr_i" slider in the plotter if I multiplied it by 10, as the max value on the plotter is "57912.00"
# Plotter's slider: "57912.00": curr_i: 57265, t_start: 1910.79, t_stop: 1920.74. Note the slider fails to update at the end, so curr_i is probably a full value.
active_epoch_session.laps.lap_start_stop_slider_values = active_epoch_session.laps.lap_start_stop_time * 10.0
active_epoch_session.laps

<Laps: {'_filename': None, '_metadata': None, 'lap_id': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42], dtype=int64), 'laps_spike_counts': array([16993,  9234, 17670,  4971,  8291,  4587, 15771,  3855,  8833,
        4628,  8411,  3730,  4228,  3332,  7126,  3506,  3984,  3548,
        5843,  3079,  4052,  2433,  3473,  2907,  5027,  2271,  3684,
        3272,  2837,  2994,  3309,  2653,  3090,  2627,  4834,  2220,
        5768,  3046,  3928,  3486, 11220,  9136], dtype=int64), 'lap_start_stop_flat_idx': array([[ 15841., 900605.],
       [ 31363., 910918.],
       [ 51216.,  68885.],
       [ 68886.,  73856.],
       [145897., 154187.],
       [162648., 167234.],
       [343909., 359679.],
       [369732., 373586.],
       [396149., 404981.],
       [412413., 417040.],
       [422331., 430741.],
       [442819., 446548.],
       [456992., 461219.

In [46]:
# active_epoch_session.flattened_spiketrains.spikes_df['t_datetime'] = pd.to_duration(active_epoch_session.flattened_spiketrains.spikes_df['t_seconds'])

# active_epoch_session.flattened_spiketrains.spikes_df['t_datetime'] = pd.to_datetime(active_epoch_session.flattened_spiketrains.spikes_df['t_seconds']).dt.strftime('%H:%M:%S')
# df1['Time'] = pd.to_datetime(df1['t_seconds']).apply(lambda x: x.replace(microsecond=0) # make integer seconds if wanted





# df2['Time'] = pd.to_datetime(df2['Time']).dt.strftime('%H:%M:%S')
active_epoch_session.flattened_spiketrains.spikes_df['t_seconds'].values[0] # 0.0161


0.0161

In [47]:
print('pix2cm: {}'.format(DataSessionLoader.pix2cm))

pix2cm: 287.7698


In [48]:
active_sess_laps = active_epoch_session.laps
# print('active_sess_laps: {}'.format(active_sess_laps))
active_sess_laps.lap_start_stop_time # [  96.0589, 6223.86  ], [ 190.4446, 6285.0087], ... these values look like 't' format, not 't_seconds'
# active_sess_laps.laps_spike_counts
# active_sess_laps.lap_start_stop_flat_idx
def get_lap_flat_indicies(lap_id):
    start_stop = active_sess_laps.lap_start_stop_flat_idx[lap_id,:] # array([ 15841., 900605.]) the start_stop time for the first lap
    return start_stop[0], start_stop[1]

def get_lap_times(lap_id):
    start_stop = active_sess_laps.lap_start_stop_time[lap_id,:] # array([ 886.4489000000001, 931.6386]) the start_stop time for the first lap
    return start_stop[0], start_stop[1]

def build_lap_specific_lists(active_epoch_session):
    # Group by the lap column:
    lap_grouped_spikes_df = active_epoch_session.flattened_spiketrains.spikes_df.groupby(['lap']) #  as_index=False keeps the original index
    
    lap_specific_subsessions = list()
    lap_specific_dataframes = list()
    lap_spike_indicies = list()
    lap_spike_t_seconds = list()
    for i in np.arange(active_epoch_session.laps.n_laps):
        curr_lap_id = active_epoch_session.laps.lap_id[i]
        #curr_flat_cell_indicies = (flat_spikes_out_dict['aclu'] == curr_cell_id) # the indicies where the cell_id matches the current one
        # print('curr_lap_id: {}'.format(curr_lap_id))
        if curr_lap_id in lap_grouped_spikes_df.groups.keys():
            curr_lap_dataframe = lap_grouped_spikes_df.get_group(curr_lap_id)
            lap_specific_dataframes.append(curr_lap_dataframe)
            lap_spike_indicies.append(curr_lap_dataframe.flat_spike_idx.values)
            lap_spike_t_seconds.append(curr_lap_dataframe.t_seconds.values)
            lap_specific_subsessions.append(active_epoch_session.time_slice(curr_lap_dataframe.t_seconds.values[0], curr_lap_dataframe.t_seconds.values[-1]))
        else:
            # curr_lap_dataframe = pd.DataFrame()
            lap_specific_dataframes.append([])
            lap_spike_indicies.append([])
            lap_spike_t_seconds.append([])
            lap_specific_subsessions.append(None)
        # curr_lap_spike_indicies = curr_lap_dataframe.flat_spike_idx.values
        # lap_spike_indicies.append(curr_lap_dataframe.flat_spike_idx.values)
        # spiketrains.append(curr_cell_dataframe[time_variable_name].to_numpy())
        # shank_ids[i] = curr_cell_dataframe['shank'].to_numpy()[0] # get the first shank identifier, which should be the same for all of this curr_cell_id
        # cell_quality[i] = curr_cell_dataframe['qclu'].mean() # should be the same for all instances of curr_cell_id, but use mean just to make sure
        # cell_type.append(curr_cell_dataframe['cell_type'].to_numpy()[0])
    return lap_specific_subsessions, lap_specific_dataframes, lap_spike_indicies, lap_spike_t_seconds
        
    
    
# curr_lap_id = 4
# curr_lap_t_start, curr_lap_t_stop = get_lap_times(curr_lap_id)
# print('lap[{}]: ({}, {})'.format(curr_lap_id, curr_lap_t_start, curr_lap_t_stop))

In [49]:
active_epoch_session.laps.lap_id

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42], dtype=int64)

In [50]:
lap_specific_subsessions, lap_specific_dataframes, lap_spike_indicies, lap_spike_t_seconds = build_lap_specific_lists(active_epoch_session)
# print('lap_spike_indicies: {}'.format(lap_spike_indicies))

# lap_spike_indicies[4] # all spike indicies that occured within the lap
# sess.laps.n_laps
lap_specific_dataframes[4]
lap_specific_subsessions[4].position.traces
np.shape(lap_specific_subsessions[4].position.traces) # (2, 1299)

Constraining to epoch with times (start: 96.27120000000001, end: 153.10500000000002)
Constraining to epoch with times (start: 190.5316, end: 220.6108)
Constraining to epoch with times (start: 298.53360000000004, end: 402.72990000000004)
Constraining to epoch with times (start: 403.10490000000004, end: 427.2926)
Constraining to epoch with times (start: 886.6108, end: 929.9599000000001)
Constraining to epoch with times (start: 982.3486, end: 1003.7444)


(2, 1299)

In [15]:
# flattened_spikes.flattened_spike_identities # array([53, 45, 45, ..., 45, 22,  2])
# flattened_spikes.flattened_sort_indicies # array([16984, 13674, 13675, ..., 16068,  7575,   580], dtype=int64)
# flattened_spikes.flattened_spike_times # array([1.6100000e-02, 2.7230000e-01, 2.8300000e-01, ..., 1.9308899e+03, 1.9316915e+03, 1.9320706e+03])
# print(flattened_spike_positions_list) # [[103.5329515  100.94484811 100.86902587 ... 215.55358368 216.0612899 217.59276516], [100.64776532  98.26058916  98.16262888 ... 139.36778275 138.94465291 139.07649496]]
# num_flattened_spikes # 19647

In [16]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_config.plotting_config.active_cells_colormap)

Based on frate_thresh, excluded neuron_ids: [ 4  7 11 33 47]


## Main Spike/Placemap plotting:

# Plot Placemaps on 3D Plot

In [17]:
t = active_epoch_pos.time
x = active_epoch_pos.x
y = active_epoch_pos.y

In [18]:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer

try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, x, y, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)

In [19]:
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter)

No extant BackgroundPlotter
Creating a new BackgroundPlotter
done.


C:\Users\Pho\repos\NeuroPy\neuropy\utils\mathutil.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (x - np.min(x, axis=axis, keepdims=True)) / np.ptp(


num_curr_tuning_curves: 30


In [51]:
## Slider with Callback Function Example:
import PhoGui
from PhoGui.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
print('done.')

active_epoch_pos.sampling_rate (Hz): 29.96976599553983
longer_spikes_window - curr_view_window_length_samples - 30689
recent_spikes_window - curr_view_window_length_samples - 299
Applying custom Pyvista theme.
done.
done.


In [52]:
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

No open BackgroundPlotter
Creating a new BackgroundPlotter
done.
[f] - Focus and zoom in on the last clicked point
shift+click - Drag to pan the rendering scene
ctrl+click - Rotate the scene in 2D


In [22]:
# ## Playback animation:
# # pl = PhoInteractivePlotter(pyvista_plotter=p)

# # ## Testing Updating the Timestamp Slider Programmatically:
# # interactive_timestamp_slider_wrapper.curr_index
# # interactive_timestamp_slider_wrapper.step_index(15)
        
# animate = InterfaceProperties(interactive_timestamp_slider_wrapper)
# # An unused constant-time callback that calls back every so often to perform updates
# p.add_callback(animate, interval=16)  # to be smooth on 60Hz

# # A checkbox that decides whether we're playing back at a constant rate or not.
# interactive_checkbox_actor = p.add_checkbox_button_widget(toggle_animation, value=False, color_on='green')

# Programmatically change the animal position trail:

In [53]:
def plot_lap_trajectory_path(ipspikesDataExplorer, curr_lap_position_traces):
    num_lap_samples = np.shape(curr_lap_position_traces)[1]
    lap_fixed_z = np.full_like(curr_lap_position_traces[0,:], 0.9)
    plot_name = 'lap_location_trail'
    # fade_values = ipspikesDataExplorer.params.active_trail_opacity_values
    # size_values = ipspikesDataExplorer.params.active_trail_size_values
    trail_fade_values = None
    size_values = None
    # trail_fade_values = np.linspace(0.0, 0.6, self.params.curr_view_window_length_samples)
    # size_values = np.linspace(0.2, 0.6, self.params.curr_view_window_length_samples) # fade from a scale of 0.2 to 0.6
    ipspikesDataExplorer.perform_plot_location_trail(plot_name, curr_lap_position_traces[0,:], curr_lap_position_traces[1,:], lap_fixed_z,
                                                 trail_fade_values=trail_fade_values, trail_point_size_values=size_values,
                                                 render=True)

In [56]:
## ACTIVE: Laps and Spikes:

# def visually_indicate_current_lap(
curr_lap_id = 4

curr_lap_t_start, curr_lap_t_stop = get_lap_times(curr_lap_id)
curr_lap_subsession = lap_specific_subsessions[curr_lap_id]
curr_lap_dataframe = lap_specific_dataframes[curr_lap_id] # the subset dataframe for this lap
curr_lap_spike_t_seconds = curr_lap_dataframe.t_seconds
curr_lap_spike_indicies = lap_spike_indicies[curr_lap_id] # all spike indicies that occured within the lap
curr_lap_position_traces = curr_lap_subsession.position.traces # the animal positions that were traversed during this lap

curr_lap_num_spikes = len(curr_lap_spike_indicies)
print('lap[{}]: ({}, {}): {} spikes.'.format(curr_lap_id, curr_lap_t_start, curr_lap_t_stop, curr_lap_num_spikes))

test_flat_times = active_epoch_session.flattened_spiketrains.flattened_spike_times
print(test_flat_times)
    
plot_lap_trajectory_path(ipspikesDataExplorer, curr_lap_position_traces)
# curr_lap_t_start
## TODO: enable showing/hiding the spikes for this data range programmatically in InteractivePlaceCellDataExplorer. 
# ipspikesDataExplorer.
# curr_lap_spike_t_seconds
# curr_lap_spike_indicies
# active_epoch_session.flattened_spiketrains.spikes_df.t_seconds.values[curr_lap_spike_indicies] # index 145937 is out of bounds for axis 0 with size 19647

lap[4]: (886.4489000000001, 931.6386): 437 spikes.
[1.6100000e-02 2.7230000e-01 2.8300000e-01 ... 1.9308899e+03 1.9316915e+03
 1.9320706e+03]


In [24]:
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_indicies, active_included_recent_only_indicies=curr_lap_spike_indicies) # index 145937 is out of bounds for axis 0 with size 19647
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds, active_included_recent_only_indicies=curr_lap_spike_t_seconds) # 
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds.values, active_included_recent_only_indicies=curr_lap_spike_t_seconds.values) # 

In [105]:
# ipspikesDataExplorer.spikes_main_historical.SetVisibility(0)

# ipspikesDataExplorer.spikes_main_historical.SetVisibility(1)
# bool(ipspikesDataExplorer.spikes_main_historical.GetVisibility())

ipspikesDataExplorer.toggle_plot_visibility('spikes_main_historical')
# toggle_visibility(ipspikesDataExplorer.spikes_main_historical)

In [106]:
# ipspikesDataExplorer.spikes_main_recent_only.SetVisibility(True)
toggle_visibility(ipspikesDataExplorer.spikes_main_recent_only)

In [107]:
# ipspikesDataExplorer.animal_location_trail.SetVisibility(0)
toggle_visibility(ipspikesDataExplorer.animal_location_trail)

In [ ]:
# ipspikesDataExplorer.animal_location_trail.SetVisibility(0)
toggle_visibility(ipspikesDataExplorer.animal_current_location_point)

# Programmatically change the animals location point indicator:

In [ ]:
# .GetPosition() returns the actor's position, which is usually (0.0, 0.0, 0.0), and does not relate to the actor's data points
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetPosition()
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetXRange() # (44.23604202270508, 245.9059600830078)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetYRange() # (135.27638244628906, 145.448974609375)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetZRange() # (1.100000023841858, 1.100000023841858)

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
print('curr_animal_point: {}'.format(curr_animal_point))
curr_animal_point # curr_animal_poin

# curr_animal_point = np.array([0, 0, 0])

# curr_animal_point = np.column_stack((self.x[active_window_sample_indicies], self.y[active_window_sample_indicies], self.z_fixed))
# ipspikesDataExplorer.on_programmatic_data_update(curr_animal_point=curr_animal_point)

curr_debug_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
ipspikesDataExplorer.perform_plot_location_point('debug_point_plot', curr_animal_point)

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # not updated
print('new curr_animal_point: {}'.format(curr_animal_point))

In [74]:
# ipspikesDataExplorer.flattened_spike_positions_list

active_epoch_session.flattened_spiketrains.time_slice(curr_lap_spike_t_seconds.values[0], curr_lap_spike_t_seconds.values[-1]).spikes_df

In [ ]:
curr_lap_dataframe

In [ ]:
active_epoch_session.time_slice